# Apêndice A.

In [1]:
!pip list

Package                 Version
----------------------- ---------
absl-py                 0.13.0
argon2-cffi             21.1.0
astunparse              1.6.3
attrs                   21.2.0
backcall                0.2.0
bleach                  4.1.0
cachetools              4.2.2
certifi                 2021.5.30
cffi                    1.14.6
charset-normalizer      2.0.4
clang                   5.0
cycler                  0.10.0
debugpy                 1.4.3
decorator               5.1.0
defusedxml              0.7.1
entrypoints             0.3
flatbuffers             1.12
gast                    0.4.0
google-auth             1.35.0
google-auth-oauthlib    0.4.6
google-pasta            0.2.0
grpcio                  1.40.0
h5py                    3.1.0
idna                    3.2
ipykernel               6.4.1
ipython                 7.27.0
ipython-genutils        0.2.0
ipywidgets              7.6.4
jedi                    0.18.0
Jinja2                  3.0.1
jsonschema              3.2.

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"

## Importações.

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import sys
import PIL
import PIL.Image
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras
from keras import layers


tf.test.gpu_device_name()

flag = 1 #treinar modelo
flag = 0 #carregar modelo !! (Comente para treinar e salvar)

2021-09-13 00:03:54.905543: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-09-13 00:03:54.906107: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-09-13 00:03:54.906120: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-09-13 00:03:54.906134: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pedro-vostro): /proc/driver/nvidia/version does not exist


## Configurações.

In [4]:
input_shape = (160,160,3)
batch = 32

vagas = 40
pontos = vagas*4

## Importação do Dataset.

In [5]:
BASE_PATH = ['../..','datasets']
DATASET_DIR=['PKLot','UFPR05']

DATASET_DIR = os.path.join(BASE_PATH[0],BASE_PATH[1],
                           DATASET_DIR[0],DATASET_DIR[1])
DATASET_DIR

'../../datasets/PKLot/UFPR05'

In [6]:
IMAGE_DIR = os.path.join(DATASET_DIR,'todas')
XML_DIR = IMAGE_DIR


IMAGE_LIST = []
XML_LIST = []
def criarLista():
    for file in os.listdir(IMAGE_DIR):
        if file.endswith(".jpg"):

            file = os.path.join(IMAGE_DIR,file)
            IMAGE_LIST.append(file)
    IMAGE_LIST.sort()

    for file in os.listdir(IMAGE_DIR):
        if file.endswith(".xml"):

            file = os.path.join(XML_DIR,file)
            XML_LIST.append(file)
    XML_LIST.sort()


criarLista()
#debug
print(IMAGE_DIR)
print(XML_DIR)
print(IMAGE_LIST[:2])
print(XML_LIST[:2])

../../datasets/PKLot/UFPR05/todas
../../datasets/PKLot/UFPR05/todas
['../../datasets/PKLot/UFPR05/todas/2013-02-22_06_05_00.jpg', '../../datasets/PKLot/UFPR05/todas/2013-02-22_06_10_00.jpg']
['../../datasets/PKLot/UFPR05/todas/2013-02-22_06_05_00.xml', '../../datasets/PKLot/UFPR05/todas/2013-02-22_06_10_00.xml']


In [7]:
BASE_PATH = ['../..','datasets']
DATASET_DIR=['PKLot','UFPR05']

DATASET_DIR = os.path.join(BASE_PATH[0],BASE_PATH[1],
                           DATASET_DIR[0],DATASET_DIR[1])

DADOS_DIR = os.path.join(DATASET_DIR,'output')
X = tf.keras.preprocessing.image_dataset_from_directory(DADOS_DIR, image_size=input_shape[:2], batch_size=batch, label_mode='binary',
                                                        shuffle=True, seed=123, color_mode='rgb', validation_split=0.3, subset = 'training')


X_val = tf.keras.preprocessing.image_dataset_from_directory(DADOS_DIR, image_size=input_shape[:2], batch_size=batch, label_mode='binary',
                                                        shuffle=True, seed=123, color_mode='rgb', validation_split=0.3, subset = 'validation')

class_names = X.class_names


print('\nClasses: {} em {}'.format(X.class_names,DADOS_DIR))

Found 166080 files belonging to 2 classes.
Using 116256 files for training.
Found 166080 files belonging to 2 classes.
Using 49824 files for validation.

Classes: ['livre', 'ocupada'] em ../../datasets/PKLot/UFPR05/output


In [8]:
val_batches = tf.data.experimental.cardinality(X_val)
X_test = X_val.take(val_batches // 5)
X_val  = X_val.skip(val_batches // 5)

print('Number of validation batches: %d' % tf.data.experimental.cardinality(X_val))
print('Number of test batches: %d' % tf.data.experimental.cardinality(X_test))



Number of validation batches: 1246
Number of test batches: 311


In [9]:
AUTOTUNE = tf.data.AUTOTUNE

X = X.prefetch(buffer_size=AUTOTUNE)
X_val = X_val.prefetch(buffer_size=AUTOTUNE)
X_test = X_test.prefetch(buffer_size=AUTOTUNE)

## Pre-processamento dos dados.

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

for image, _ in X.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

2021-09-13 00:04:01.927571: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


## Testando os Labels carregadas da base de treinamento.

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in X.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(int(labels[i]))
        plt.axis("off")

### Exemplo de Imagem não segmentada.

In [ ]:
sort = np.random.randint(len(IMAGE_LIST))

PIL.Image.open(IMAGE_LIST[sort])

## Transfer Learning

### Normalizar as imagens para o padrão do modelo da mobilenet_v2

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

### Criar o modelo base.

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=input_shape,
                                               include_top=False,
                                               weights='imagenet')

base_model.trainable = False
#base_model.summary()

image_batch, label_batch = next(iter(X))
feature_batch = base_model(image_batch)
feature_batch.shape

## Construindo o modelo e compilando.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

inputs = tf.keras.Input(shape=(160, 160, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False) # O modelo contem uma camada de Normalização. Não destruir os pesos aprendidos
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)
model = tf.keras.Model(inputs, outputs)

base_learning_rate = 0.0001

model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy,
              metrics=['accuracy'])

model.summary()

# def modelo():
#     model = keras.Sequential(
#     [
#         layers.InputLayer(input_shape),
#         layers.Conv2D(filters=32,kernel_size=5,strides=2, activation="relu"),
#         layers.Conv2D(32, 3, activation="relu"),
#         layers.MaxPooling2D(pool_size=3),
#         layers.BatchNormalization(),
        
#         layers.Conv2D(32,3, activation="relu"),
#         layers.Conv2D(32,3, activation="relu"),
#         layers.MaxPooling2D(2),
#         layers.BatchNormalization(),
        
#         layers.Flatten(),
#         layers.Dense(48,activation='relu'),
#         layers.Dropout(0.3),
#         layers.Dense(48,activation='relu'),
#         layers.Dropout(0.3),
#         layers.Dense(1, activation="sigmoid"),
#     ]
# )

#     return model

# #model = modelo()

# model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
# model.summary()
# keras.utils.plot_model(model, "./images/arquitetura.png", show_shapes=True)

## Treinando o modelo.

In [ ]:
%%time

if flag:
    
    loss0, accuracy0 = model.evaluate(X_val)
    
    print("initial loss: {:.2f}".format(loss0))
    print("initial accuracy: {:.2f}".format(accuracy0))
    
    history = model.fit(X, epochs=5, validation_data=X_val)
    
    
else:
    #LOAD_DIR = os.path.split(DATASET_DIR)[0] # Separa (head,tail) -> Indice [0] = head
    LOAD_DIR = os.path.join(DATASET_DIR,'model')

    model = keras.models.load_model(LOAD_DIR)
    

## Métricas

In [ ]:
if flag:
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    loss = history.history['loss']
    val_loss = history.history['val_loss']

    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()),1])
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0,0.1])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

## Validação do modelo treinado. (testando na base de dados de validação)

In [ ]:
if flag:
    score = model.evaluate(X_test, verbose=2)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

## Salvando o modelo treinado.

In [ ]:
if flag:
    SAVE_DIR = os.path.split(DATASET_DIR)[0] # Separa (head,tail) -> Indice [0] = head
    SAVE_DIR = os.path.join(DATASET_DIR,'model')
    model.save(
        SAVE_DIR,
        overwrite=False,
        include_optimizer=True,
        save_format=None,
        signatures=None,
        options=None,
        save_traces=True,
    )

## Validação individual (random) na base de treinamento

In [ ]:
X = tf.keras.preprocessing.image_dataset_from_directory(DADOS_DIR, image_size=input_shape[:2], batch_size=batch, label_mode='binary',
                                                        shuffle=True, seed=123, color_mode='rgb', validation_split=0.3, subset = 'training')


X_val = tf.keras.preprocessing.image_dataset_from_directory(DADOS_DIR, image_size=input_shape[:2], batch_size=batch, label_mode='binary',
                                                        shuffle=True, seed=123, color_mode='rgb', validation_split=0.3, subset = 'validation')

In [ ]:
path = X.file_paths[np.random.randint(len(X.file_paths))]
img = keras.preprocessing.image.load_img(path, target_size=input_shape[:2])
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array,  0)

predictions = model.predict(img_array)

predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

score = predictions[0]

print(
    'Vaga livre: %.2f porcento.\nVaga ocupada %.2f porcento.'
    % (100 * (1 - score), 100 * score)
)

#plot = plt.imread(path)
#plt.imshow(img,cmap='gray')
plt.imshow(img)

## Validação individual (random) no Banco de testes

In [ ]:
path = X_val.file_paths[np.random.randint(len(X_val.file_paths))]
img = keras.preprocessing.image.load_img(path, target_size=input_shape[:2])
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array,  0)

predictions = model.predict(img_array)

#predictions = tf.nn.sigmoid(predictions)
#predictions = tf.where(predictions < 0.5, 0, 1)

score = predictions[0]

print(
    'Vaga livre: %.2f porcento.\nVaga ocupada %.2f porcento.'
    % (100 * (1 - score), 100 * score)
)

#plot = plt.imread(path)
plt.imshow(img)


In [ ]:
#Retrieve a batch of images from the test set
image_batch, label_batch = X_test.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch).flatten()

# Apply a sigmoid since our model returns logits
predictions = tf.nn.sigmoid(predictions)
predictions = tf.where(predictions < 0.5, 0, 1)

for i in range (len(predictions)):
    print('\nPredictions = {}, {} = Label'.format(predictions[i].numpy(),label_batch[i].tolist()))


plt.figure(figsize=(25, 25))
for i in range(32):
  ax = plt.subplot(8, 4, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(class_names[predictions[i]])
  plt.axis("off")

## CARREGANDO XML E CRIANDO OS PONTOS

In [ ]:
tree = ET.parse(XML_LIST[0])
root = tree.getroot()

pts = np.empty(0,np.int32)
vaga = np.empty(0,np.int32)

for neighbor in root.iter('point'):
    #print(neighbor.attrib)
    x,y = neighbor.attrib.values()
    x = int(x)
    y = int(y)
    a = [x,y]
    pts = np.append(pts,[x,y])
    
for neighbor in root.iter('space'):
    #print(neighbor.attrib)
    if(len(neighbor.attrib.values())==2):
        _,occupied = neighbor.attrib.values()
        occupied = int(occupied)
    if(len(neighbor.attrib.values())==1):
        occupied=0
        
    vaga = np.append(vaga,occupied)

#debug
print(pts[0:4])
print(vaga[0:40])

paresXY = np.array(np.zeros((pontos,2)),np.int32) 
j=0
for i in range(pontos):
    paresXY[i] = pts[j:j+2]
    j = j+2

#debug
paresXY[0:4]

In [ ]:
def crop(j=0,i=0):
    rect = cv2.boundingRect(paresXY[j:j+4])
    x,y,w,h = rect
    croped = im[y:y+h, x:x+w].copy()

    pts = paresXY[j:j+4] - paresXY[j:j+4].min(axis=0)
    mask = np.zeros(croped.shape[:2], croped.dtype)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

    ## (3) do bit-op
    dst = cv2.bitwise_and(croped, croped, mask=mask) #background preto

    bg = np.ones_like(croped, np.uint8)*255
    cv2.bitwise_not(bg,bg, mask=mask)
    dst2 = bg+ dst #background branco
    
    image = cv2.resize(dst2, input_shape[:2], interpolation = cv2.INTER_CUBIC)
    
    img = image.copy()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array,  0)

    predictions = model.predict(img_array)
     
    score = predictions[0]
    score = float(score)
    return score
    

In [ ]:
def juntar(im2,im,scale=80):
    
    img = np.hstack((im2, im))
    
    scale_percent = scale # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)  
    return resized

In [ ]:
nVagas = 3
nVagas = vagas - nVagas

## Varias imagens.

In [ ]:
for i in range (1): #só repetição
    
    sort = np.random.randint(len(IMAGE_LIST))
    im = cv2.imread(IMAGE_LIST[sort])
    im2 = im.copy()
    if im is None:
        sys.exit("A imagem não foi carregada.")

    j=0
    contador = 0
    for i in range(nVagas,len(paresXY)//4):

        score = crop(j,i)
        #print(score)
        if((score)<0.0):
            cv2.polylines(im,[paresXY[j:j+4]],True,(0,255,255),2)
            contador = contador +1
            #print(score)
        j=j+4
        
        

    cv2.putText(im,str(contador),(10,700), cv2.FONT_HERSHEY_SIMPLEX, 4,(255,255,255),5,cv2.LINE_AA)
    img_junta = juntar(im2,im,70)

    cv2.imshow(IMAGE_LIST[sort], img_junta)
    k = cv2.waitKey(3000)
    cv2.destroyAllWindows()
# if k == ord('q'):
#     cv2.destroyAllWindows()

#image = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
#plt.imshow(image)

## Única Imagem.

In [ ]:
sort = np.random.randint(len(IMAGE_LIST))
im = cv2.imread(IMAGE_LIST[350])
im2 = im.copy()
if im is None:
    sys.exit("A imagem não foi carregada.")

j=0
contador = 0
for i in range(nVagas,len(paresXY)//4):

    score = crop(j,i)
    #print(score)
    if((score)<0.0):
        cv2.polylines(im,[paresXY[j:j+4]],True,(0,255,255),2)
        contador = contador +1
        #print(score)
    j=j+4



cv2.putText(im,str(contador),(10,700), cv2.FONT_HERSHEY_SIMPLEX, 4,(255,255,255),5,cv2.LINE_AA)
img_junta = juntar(im2,im,70)

cv2.imshow(IMAGE_LIST[sort], im)
cv2.waitKey(0)
cv2.destroyAllWindows()

#image = cv2.cvtColor(im,cv2.COLOR_BGR2RGB)
#plt.imshow(image)

## VIDEO

## Debug 

In [ ]:
adsasad

In [ ]:
!pip list

In [ ]:
!pip install opencv-python==4.4.0.44

In [ ]:
!pip uninstall PyQt5 PyQt5-Qt5 PyQt5-sip opencv-python -y

In [ ]:
!pip uninstall QtPy -y

In [ ]:
!pip install PyQt5

In [ ]:
!pip install PyQt

In [ ]:
pip install --no-binary opencv-python opencv-python